<a href="https://colab.research.google.com/github/Piligrimich/Piligrimich/blob/main/GoogLeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import random_split

import torchvision.datasets as dset

from torchvision import transforms

import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

train_data = dset.CIFAR100('./',download=True,train=True,transform=transform)
test_data = dset.CIFAR100('./',download=True,train=False,transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
batch_size = 64

data_size = train_data.data.shape[0]
validation_split = .2
split = int(np.floor(data_size*validation_split))

indices = list(range(data_size))

np.random.shuffle(indices)

train_indices, val_indices = indices[split:],indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(train_data,batch_size=batch_size,sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(train_data,batch_size=batch_size,sampler=val_sampler)


In [4]:
from torchvision import models

In [5]:
model = models.googlenet(pretrained=True)

In [ ]:
print(model)

In [23]:
num_ftrs = model.fc.in_features

for param in model.parameters():
  param.requires_grad = False
  
model.fc = nn.Linear(num_ftrs,100)

In [7]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs,100)

In [10]:
num_ftrs

1024

In [7]:
for param in model.parameters():
  param.requires_grad = False

In [20]:
for name, child in model.named_children():
  print(name)

conv1
maxpool1
conv2
conv3
maxpool2
inception3a
inception3b
maxpool3
inception4a
inception4b
inception4c
inception4d
inception4e
maxpool4
inception5a
inception5b
avgpool
dropout
fc


In [8]:
if torch.cuda.is_available:
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

print(device)

cuda


In [9]:
def fit(model,train_loader,val_loader,loss,optimizer,tol_epochs,accuracy_tol):
  
    epoch = 0
   
    
    #loss_sum = 0
    keep_training = True
    last_accuracies =[]

    while keep_training == True:
      model.train()
      correct_samples = 0
      total_samples = 0

      for x,y in train_loader:
        x,y = x.to(device),y.to(device)
        
        outputs = model(x)
        loss_value = loss(outputs,y)

        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()

        _,indices = torch.max(outputs,dim=1)
        correct_samples += torch.sum(indices == y)
        total_samples += y.shape[0]
        #loss_sum += loss_value
      val_accuracy = validationAccuracy(model,val_loader)
      
      print("Epoch: %f  Accuracy %f  Val accuracy %f "%(epoch+1,(correct_samples/total_samples),val_accuracy))
      epoch+=1
      last_accuracies.append(val_accuracy)

      if len(last_accuracies) > tol_epochs:
        last_accuracies.pop(0)
      
      if len(last_accuracies) == tol_epochs:
        accuracy_difference = max(last_accuracies)-min(last_accuracies)
        if accuracy_difference <= accuracy_tol:
          keep_training = False


def validationAccuracy(model,val_loader):
  model.eval()
  correct_samples = 0
  total_samples = 0
  for i_step,(x,y) in enumerate(val_loader):
    x,y = x.to(device),y.to(device)
    predicts = model(x)

    
    _, indices = torch.max(predicts,dim=1)
    correct_samples += torch.sum(indices == y)
    total_samples += y.shape[0]

  return correct_samples/total_samples

In [10]:
optimizer = torch.optim.Adam(model.parameters())
loss = nn.CrossEntropyLoss()

In [17]:
train_data.data.shape

(50000, 32, 32, 3)

In [11]:
model.to(device)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [15]:
fit(model,train_loader,val_loader,loss,optimizer,accuracy_tol=0.02,tol_epochs=10)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Epoch: 1.000000  Accuracy 0.248250  Val accuracy 0.343100 
Epoch: 2.000000  Accuracy 0.290675  Val accuracy 0.395800 
Epoch: 3.000000  Accuracy 0.331508  Val accuracy 0.389700 
Epoch: 4.000000  Accuracy 0.364094  Val accuracy 0.433700 
Epoch: 5.000000  Accuracy 0.393310  Val accuracy 0.441800 
Epoch: 6.000000  Accuracy 0.418458  Val accuracy 0.470500 
Epoch: 7.000000  Accuracy 0.440093  Val accuracy 0.454600 
Epoch: 8.000000  Accuracy 0.460066  Val accuracy 0.453400 
Epoch: 9.000000  Accuracy 0.477964  Val accuracy 0.481000 
Epoch: 10.000000  Accuracy 0.495000  Val accuracy 0.472000 
Epoch: 11.000000  Accuracy 0.510882  Val accuracy 0.472300 
Epoch: 12.000000  Accuracy 0.525581  Val accuracy 0.471900 
Epoch: 13.000000  Accuracy 0.539000  Val accuracy 0.474600 
Epoch: 14.000000  Accuracy 0.552080  Val accuracy 0.483700 
Epoch: 15.000000  Accuracy 0.564513  Val accuracy 0.486200 
Epoch: 16.000000  Accuracy 0.576317  Val accuracy 0.471900 
Epoch: 17.000000  Accuracy 0.587150  Val accuracy

In [12]:
fit(model,train_loader,val_loader,loss,optimizer,accuracy_tol=0.02,tol_epochs=10)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Epoch: 1.000000  Accuracy 0.251850  Val accuracy 0.360000 
Epoch: 2.000000  Accuracy 0.402025  Val accuracy 0.397300 
Epoch: 3.000000  Accuracy 0.473175  Val accuracy 0.436800 
Epoch: 4.000000  Accuracy 0.530300  Val accuracy 0.444900 
Epoch: 5.000000  Accuracy 0.584650  Val accuracy 0.479400 
Epoch: 6.000000  Accuracy 0.623775  Val accuracy 0.487700 
Epoch: 7.000000  Accuracy 0.658150  Val accuracy 0.500800 
Epoch: 8.000000  Accuracy 0.696500  Val accuracy 0.502400 
Epoch: 9.000000  Accuracy 0.726100  Val accuracy 0.490400 
Epoch: 10.000000  Accuracy 0.754525  Val accuracy 0.498600 
Epoch: 11.000000  Accuracy 0.776625  Val accuracy 0.501500 
Epoch: 12.000000  Accuracy 0.801125  Val accuracy 0.501800 
Epoch: 13.000000  Accuracy 0.819550  Val accuracy 0.507000 
Epoch: 14.000000  Accuracy 0.833175  Val accuracy 0.505500 
Epoch: 15.000000  Accuracy 0.849200  Val accuracy 0.497400 


In [ ]:
def precision(model,data):
  model.eval()
  TP = []
  FP = []
  for X,Y in data:
    X = X.to(device)
    pred = model(X)
    _, indices = torch.max(pred,axis=1)
    for i in range(len(indices)):
      if indices[i] == Y[i]:
        TP.append(indices[i])
      if indices

    correct_samples = torch.sum(indices==Y)

    indices = indices.detach().numpy()


In [13]:
import sklearn.metrics

In [ ]:
sklearn.metrics.accuracy_score()

In [34]:
#def accuracy(model,data):
model.eval()
correct_samples = 0
total_samples = 0
for i_step,(x,y) in enumerate(val_loader):
  x,y = x.to(device),y.to(device)
  predicts = model(x)

  
  _, indices = torch.max(predicts,dim=1)
  correct_samples += torch.sum(indices == y)
  total_samples += y.shape[0]
  indices = indices.to('cpu')
  indices = indices.detach().numpy()
  accuer = sklearn.metrics.accuracy_score(y.to('cpu').detach().numpy(),indices)
  #precision = sklearn.metrics.precision_score(y.to('cpu').detach().numpy(),indices)
  clasif_report = sklearn.metrics.classification_report(y.to('cpu').detach().numpy(),indices)
  #accuer = accuer.detach().numpy()
  print('Самодельный Accuracy: ',correct_samples/total_samples)
  print('Метрика sklearn: ',accuer)
  print('clasif_report: ',clasif_report)
 

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318:

Самодельный Accuracy:  tensor(0.5312, device='cuda:0')
Метрика sklearn:  0.53125
clasif_report:                precision    recall  f1-score   support

           0       1.00      0.67      0.80         3
           2       0.00      0.00      0.00         1
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           9       0.00      0.00      0.00         0
          12       0.50      1.00      0.67         1
          13       1.00      1.00      1.00         1
          14       1.00      1.00      1.00         1
          16       0.00      0.00      0.00         0
          18       0.00      0.00      0.00         0
          19       0.00      0.00      0.00         1
          20       1.00      1.00      1.00         1
          21       1.00      1.00      1.00         1
          23       0.00      0.00      0.00         1
          26       0.50      1.00      0.67         2
          27       0.00      0.00     

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318:

Самодельный Accuracy:  tensor(0.4933, device='cuda:0')
Метрика sklearn:  0.4375
clasif_report:                precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           2       0.00      0.00      0.00         0
           5       0.50      1.00      0.67         1
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         1
          10       1.00      1.00      1.00         1
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0
          15       1.00      0.25      0.40         4
          16       1.00      1.00      1.00         1
          18       0.00      0.00      0.00         0
          19       0.75      0.75      0.75         4
          20       1.00      1.00      

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318:

...

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318:

Самодельный Accuracy:  tensor(0.4980, device='cuda:0')
Метрика sklearn:  0.46875
clasif_report:                precision    recall  f1-score   support

           2       0.00      0.00      0.00         1
           4       0.50      1.00      0.67         1
           5       1.00      0.50      0.67         2
           9       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         1
          12       1.00      1.00      1.00         1
          14       0.00      0.00      0.00         1
          16       0.00      0.00      0.00         0
          18       0.00      0.00      0.00         1
          20       1.00      1.00      1.00         1
          21       1.00      1.00      1.00         1
          22       0.50      1.00      0.67         1
          23       1.00      1.00      1.00         1
          24       1.00      0.50      0.67         2
          26       0.25      1.00     

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318:

Самодельный Accuracy:  tensor(0.4971, device='cuda:0')
Метрика sklearn:  0.46875
clasif_report:                precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         1
           5       1.00      1.00      1.00         1
           6       0.00      0.00      0.00         1
           7       1.00      1.00      1.00         2
           8       0.50      1.00      0.67         1
          10       1.00      1.00      1.00         1
          11       0.00      0.00      0.00         2
          12       1.00      0.33      0.50         3
          15       1.00      1.00      1.00         1
          16       1.00      1.00      1.00         1
          21       0.00      0.00      0.00         0
          22       1.00      0.50     

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

In [15]:
accuracy(model,val_loader)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


TypeError: ignored

In [ ]:
'''batch_size = 64
validation_split =.2
split = int(np.floor(train_data.data.size[0]*validation_split))

train_subset,val_subset = random_split(
    train_data,[train_data.data.size[0]-split, split]
)

train_loader = torch.utils.data.DataLoader(train_subset,batch_size=batch_size,shuffle=true)
'''

In [ ]:
train_data.data.shape


(50000, 32, 32, 3)

In [ ]:
class Inception(nn.Module):
  def __init__(self,in_c,ch1x1,ch3x3one,ch3x3,ch5x5one,ch5x5,pool_proj):
    super(Inception,self).__init__()
    conv_block = BasicConv2d
    self.branch1 = conv_block(in_c,ch1x1,kernel_size=1)

    self.branch2 = nn.Sequential(
      conv_block(in_c,ch3x3one,kernel_size=1),
      conv_block(ch3x3one,ch3x3,kernel_size=3, padding=1)
    )

    self.branch3 = nn.Sequential(
        conv_block(in_c,ch5x5one,kernel_size=1),
        conv_block(ch5x5one,ch5x5,kernel_size=3,padding=1)# ТУТ НА GITHUB ГОВОРИТЬСЯ ИСПОЛЬЗОВАТЬ 3x3
    )                                                     # НО В ИДЕАЛЕ НУЖНО ИСПОЛЬЗОВАТЬ 5x5 и padding=2

    self.branch4 = nn.Sequential(
        nn.MaxPool2d(kernel_size=3,stride=1,padding=1,ceil_mode=True),
        conv_block(in_c,pool_proj,kernel_size=1)
    )

  def forward(self,x):
    branch1 = self.branch1(x)
    branch2 = self.branch2(x)
    branch3 = self.branch3(x)
    branch4 = self.branch4(x)
    outputs = [branch1,branch2,branch3,branch4]
    return torch.cat(outputs,1)


In [ ]:
class BasicConv2d(nn.Module):
  def __init__(self,in_c,out_c,**kwargs):
    super().__init__()
    self. conv = nn.Conv2d(in_c,out_c,bias=False,**kwargs)
    self.bn = nn.BatchNorm2d(out_c,eps=0.001)

  def forward(self,x):
    x = self.conv(x)
    x = self.bn(x)
    return F.relu(x,inplace=True)

In [ ]:
class GoogLeNet(nn.Module):
  def __init__(self,
              num_classes: int = 100,
              aux_logits: bool = True,
              transform_input: bool = False,
              #init_weights: Optional[bool] = None,
              #blocks: Optional[List[Callable[..., nn.Module]]] = None,
              dropout: float = 0.2,
              dropout_aux: float = 0.7,
               ) -> None:
      super().__init__()
      #if blocks is None:
      blocks = [BasicConv2d, Inception, InceptionAux]
      conv_block = blocks[0]
      inception_block = blocks[1]
      inception_aux_block = blocks[2]

      self.aux_logits = aux_logits
      self.transform_input = transform_input

      self.conv1 = conv_block(3, 64, kernel_size=7, stride=2, padding=3)
      self.maxpool1 = nn.MaxPool2d(3, stride=2, ceil_mode=True)
      self.conv2 = conv_block(64, 64, kernel_size=1)
      self.conv3 = conv_block(64, 192, kernel_size=3, padding=1)
      self.maxpool2 = nn.MaxPool2d(3, stride=2, ceil_mode=True)

      self.inception3a = inception_block(192, 64, 96, 128, 16, 32, 32)
      self.inception3b = inception_block(256, 128, 128, 192, 32, 96, 64)
      self.maxpool3 = nn.MaxPool2d(3, stride=2, ceil_mode=True)

      self.inception4a = inception_block(480, 192, 96, 208, 16, 48, 64)
      self.inception4b = inception_block(512, 160, 112, 224, 24, 64, 64)
      self.inception4c = inception_block(512, 128, 128, 256, 24, 64, 64)
      self.inception4d = inception_block(512, 112, 144, 288, 32, 64, 64)
      self.inception4e = inception_block(528, 256, 160, 320, 32, 128, 128)
      self.maxpool4 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

      self.inception5a = inception_block(832, 256, 160, 320, 32, 128, 128)
      self.inception5b = inception_block(832, 384, 192, 384, 48, 128, 128)

      if aux_logits:
          self.aux1 = inception_aux_block(512, num_classes, dropout=dropout_aux)
          self.aux2 = inception_aux_block(528, num_classes, dropout=dropout_aux)
      else:
          self.aux1 = None  # type: ignore[assignment]
          self.aux2 = None  # type: ignore[assignment]

      self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
      self.dropout = nn.Dropout(p=dropout)
      self.fc = nn.Linear(1024, num_classes)